In [8]:
from enum import Enum
import numpy as np
from typing import List, Dict, Callable
import math

In [9]:
# Định nghĩa danh sách các yếu tố của tam giác
class TriangleElementType(Enum):
    a_edge = 1
    b_edge = 2
    c_edge = 3
    A_corner = 4
    B_corner = 5
    C_corner = 6
    S = 7
    p = 8
    hA = 9
    hB = 10
    hC = 11

In [10]:
class TriangleComponentType(Enum):
    Element = 1,
    Formula = 2

In [11]:
# Class cơ sở về 1 đỉnh của mạng ngữ nghĩa:
#  Yếu tố (a, b, c,..)
#  Công thức (A+B+C=Pi, ...)
class TriangleComponent:
  def __init__(self, name, label, componentType: TriangleComponentType):
    self.name = name
    self.label = label
    self.componentType = componentType


In [12]:
# Class định nghĩa về 1 yếu tố của tam giác.
class TriangleElement(TriangleComponent):
  def __init__(self, label, elementType: TriangleElementType):
    super().__init__(elementType.name, label, componentType= TriangleComponentType.Element)
    self.elementType = elementType
    self.value = None
  def __str__(self):
    return self.elementType.name  + "(" + str(self.value) + ")"

In [13]:
# Class định nghĩa công thức:
#   name, label
#   dependencyElementTypes (các yếu tố cấu thành công thức)
#   inferFunc (hàm suy luận)
class TriangleFormular(TriangleComponent):
  def __init__(self, name, label, dependencyElementTypes: List[TriangleElementType],
               inferFunc: Callable[[Dict[TriangleElementType, TriangleElement]], TriangleElementType | None] = None):
    super().__init__(name, label, componentType= TriangleComponentType.Formula)
    self.dependencyElementTypes = dependencyElementTypes
    self.inferFunc = inferFunc

In [31]:
# Định nghĩa mạng tri thức cho bài toán tam giác
class TriangleKnowledgeNetwork:
    def __init__(self, formulas: List[TriangleFormular]):
      self.formulas = formulas

    def add_formula(self, newFormula: TriangleFormular):
      self.formulas.append(newFormula)

    def propagationActivationAlg(self, initElements:  Dict[TriangleElementType, TriangleElement], processLog):
      # Matrix n hàng (tương ứng m yếu tố của tam giác), m cột (tương ứng m công thức)
      # Giá trị:
      #   0 : yếu tố và công thức không có liên quan với nhau
      #   -1: yếu tố và công thức có liên quan và yếu tố chưa có giá trị
      #   1 : yếu tố và công thức có liên quan và yếu tố đã có giá trị
      matrix = np.zeros((len(TriangleElementType), len(self.formulas)))

      # dict tất cả yếu tố của tam giác
      fullyElementDict: Dict[TriangleElementType, TriangleElement] = {}

      # khởi tạo giá trị của matrix và fullyElementDict
      for i, elementType in enumerate(TriangleElementType):
        if elementType in initElements:
          fullyElementDict[elementType] = initElements[elementType]
        else:
          fullyElementDict[elementType] = TriangleElement(elementType.name, elementType)

        for j, formula in enumerate(self.formulas):
          if elementType in formula.dependencyElementTypes:
            if elementType in initElements and initElements[elementType].value is not None :
              matrix[i, j] = 1
            else:
              matrix[i, j] = -1
      if processLog:
        print('Initial..')
        print(matrix)
        for elementType in TriangleElementType:
          print(fullyElementDict[elementType])
        # Start
        print('Start...')

      while any(fullyElementDict[element].value is None for element in fullyElementDict):
        isActive = False
        for j, formula in enumerate(self.formulas):
          # Kiểm tra công thức có thể được kích hoạt
          column_j = [row[j] for row in matrix]
          countNoValue = column_j.count(-1)
          if countNoValue == 1:
            isActive = True
            if processLog:
              print('Active the formula '+ formula.label)
            activeElementType = formula.inferFunc(fullyElementDict)
            if activeElementType is not None:
              # update matrix
              if processLog:
                print('Update element ' + activeElementType.name)
              for i, elementType in enumerate(TriangleElementType):
                if elementType == activeElementType:
                  for k in range(0, len(self.formulas)):
                    if matrix[i, k] == -1:
                      matrix[i, k] = 1
        if isActive == False:
           break
      if processLog:
        print('Finish.')
      return matrix, fullyElementDict


# Danh sách các công thức:

# 1. Công thức tổng 3 góc của 1 tam giác: A + B + C = Pi (π)

In [15]:
# A + B + C = Pi (π)
def SumCorners(currentElements:  Dict[TriangleElementType, TriangleElement]):
  A_corner_ele = currentElements[TriangleElementType.A_corner]
  B_corner_ele = currentElements[TriangleElementType.B_corner]
  C_corner_ele = currentElements[TriangleElementType.C_corner]

  if A_corner_ele is None or B_corner_ele is None or C_corner_ele is None :
    return None

  countEleNoVal = sum([1 for x in [A_corner_ele, B_corner_ele, C_corner_ele] if x.value is None])

  if countEleNoVal != 1:
    return None

  if A_corner_ele.value is None and B_corner_ele.value is not None and C_corner_ele.value is not None :
    A_corner_ele.value = math.pi - B_corner_ele.value - C_corner_ele.value
    return TriangleElementType.A_corner
  if B_corner_ele.value is None and A_corner_ele.value is not None and C_corner_ele.value is not None :
    B_corner_ele.value = math.pi - A_corner_ele.value - C_corner_ele.value
    return TriangleElementType.B_corner
  if C_corner_ele.value == None and A_corner_ele.value != None and B_corner_ele.value != None :
    C_corner_ele.value = math.pi - A_corner_ele.value - B_corner_ele.value
    return TriangleElementType.C_corner

  return None

# 2. Công thức tính diện tích tam giác từ 1 cạnh và đường cao tương ứng. S = 1/2 a.h

In [16]:
# S = 1/2 a. hA
def AreaFromHeightA(currentElements:  Dict[TriangleElementType, TriangleElement]):
  a_edge_ele = currentElements[TriangleElementType.a_edge]
  hA_ele = currentElements[TriangleElementType.hA]
  S_ele = currentElements[TriangleElementType.S]

  if a_edge_ele is None or hA_ele is None or S_ele is None:
    return None

  countEleNoVal = sum([1 for x in [a_edge_ele, hA_ele, S_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if S_ele.value is None :
    S_ele.value = 0.5 * a_edge_ele.value * hA_ele.value
    return TriangleElementType.S
  if a_edge_ele.value is None :
    a_edge_ele.value = 2*S_ele.value/hA_ele.value
    return TriangleElementType.a_edge
  if hA_ele.value is None :
    hA_ele.value = 2*S_ele.value/a_edge_ele.value
    return TriangleElementType.hA
  return None

In [17]:
# S = 1/2 b. hB
def AreaFromHeightB(currentElements:  Dict[TriangleElementType, TriangleElement]):
  b_edge_ele = currentElements[TriangleElementType.b_edge]
  hB_ele = currentElements[TriangleElementType.hB]
  S_ele = currentElements[TriangleElementType.S]

  if b_edge_ele is None or hB_ele is None or S_ele is None:
    return None

  countEleNoVal = sum([1 for x in [b_edge_ele, hB_ele, S_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if S_ele.value is None :
    S_ele.value = 0.5 * b_edge_ele.value * hB_ele.value
    return TriangleElementType.S
  if b_edge_ele.value is None :
    b_edge_ele.value = 2*S_ele.value/hB_ele.value
    return TriangleElementType.b_edge
  if hB_ele.value is None :
    hB_ele.value = 2*S_ele.value/b_edge_ele.value
    return TriangleElementType.hB
  return None

In [18]:
# S = 1/2 c. hC
def AreaFromHeightC(currentElements:  Dict[TriangleElementType, TriangleElement]):
  c_edge_ele = currentElements[TriangleElementType.c_edge]
  hC_ele = currentElements[TriangleElementType.hC]
  S_ele = currentElements[TriangleElementType.S]

  if c_edge_ele is None or hC_ele is None or S_ele is None:
    return None

  countEleNoVal = sum([1 for x in [c_edge_ele, hC_ele, S_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if S_ele.value is None :
    S_ele.value = 0.5 * c_edge_ele.value * hC_ele.value
    return TriangleElementType.S
  if c_edge_ele.value is None :
    c_edge_ele.value = 2*S_ele.value/hC_ele.value
    return TriangleElementType.c_edge
  if hC_ele.value is None :
    hC_ele.value = 2*S_ele.value/c_edge_ele.value
    return TriangleElementType.hC
  return None

# 3. Công thức tính diện tích tam giác từ 2 cạnh tam giác và góc giữa 2 cạnh đó. S = 1/2 bc.SinA

In [19]:
# S = 1/2 bc.sinA
def AreaFromSinA(currentElements:  Dict[TriangleElementType, TriangleElement]):
  b_edge_ele = currentElements[TriangleElementType.b_edge]
  c_edge_ele = currentElements[TriangleElementType.c_edge]
  A_corner_ele = currentElements[TriangleElementType.A_corner]
  S_ele = currentElements[TriangleElementType.S]

  if b_edge_ele is None or c_edge_ele is None or A_corner_ele is None or S_ele is None:
    return None

  countEleNoVal = sum([1 for x in [b_edge_ele, c_edge_ele, A_corner_ele, S_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if S_ele.value is None :
    S_ele.value = 0.5 * b_edge_ele.value * c_edge_ele.value * math.sin(A_corner_ele.value)
    return TriangleElementType.S
  if b_edge_ele.value is None :
    b_edge_ele.value = (2*S_ele.value)/(c_edge_ele.value * math.sin(A_corner_ele.value))
    return TriangleElementType.b_edge
  if c_edge_ele.value is None :
    c_edge_ele.value = (2*S_ele.value)/(b_edge_ele.value * math.sin(A_corner_ele.value))
    return TriangleElementType.c_edge
  if A_corner_ele.value is None :
    sinA = (2*S_ele.value)/(b_edge_ele.value * c_edge_ele.value)
    A_corner_ele.value = math.asin(sinA)
    return TriangleElementType.A_corner
  return None

In [20]:
# S = 1/2 ac.sinB
def AreaFromSinB(currentElements:  Dict[TriangleElementType, TriangleElement]):
  a_edge_ele = currentElements[TriangleElementType.a_edge]
  c_edge_ele = currentElements[TriangleElementType.c_edge]
  B_corner_ele = currentElements[TriangleElementType.B_corner]
  S_ele = currentElements[TriangleElementType.S]

  if a_edge_ele is None or c_edge_ele is None or B_corner_ele is None or S_ele is None:
    return None

  countEleNoVal = sum([1 for x in [a_edge_ele, c_edge_ele, B_corner_ele, S_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if S_ele.value is None :
    S_ele.value = 0.5 * a_edge_ele.value * c_edge_ele.value * math.sin(B_corner_ele.value)
    return TriangleElementType.S
  if a_edge_ele.value is None :
    a_edge_ele.value = (2*S_ele.value)/(c_edge_ele.value * math.sin(B_corner_ele.value))
    return TriangleElementType.a_edge
  if c_edge_ele.value is None :
    c_edge_ele.value = (2*S_ele.value)/(a_edge_ele.value * math.sin(B_corner_ele.value))
    return TriangleElementType.c_edge
  if B_corner_ele.value is None :
    sinB = (2*S_ele.value)/(a_edge_ele.value * c_edge_ele.value)
    B_corner_ele.value = math.asin(sinB)
    return TriangleElementType.B_corner
  return None

In [21]:
# S = 1/2 ab.sinC
def AreaFromSinC(currentElements:  Dict[TriangleElementType, TriangleElement]):
  a_edge_ele = currentElements[TriangleElementType.a_edge]
  b_edge_ele = currentElements[TriangleElementType.b_edge]
  C_corner_ele = currentElements[TriangleElementType.C_corner]
  S_ele = currentElements[TriangleElementType.S]

  if a_edge_ele is None or b_edge_ele is None or C_corner_ele is None or S_ele is None:
    return None

  countEleNoVal = sum([1 for x in [a_edge_ele, b_edge_ele, C_corner_ele, S_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if S_ele.value is None :
    S_ele.value = 0.5 * a_edge_ele.value * b_edge_ele.value * math.sin(C_corner_ele.value)
    return TriangleElementType.S
  if a_edge_ele.value is None :
    a_edge_ele.value = (2*S_ele.value)/(b_edge_ele.value * math.sin(C_corner_ele.value))
    return TriangleElementType.a_edge
  if b_edge_ele.value is None :
    b_edge_ele.value = (2*S_ele.value)/(a_edge_ele.value * math.sin(C_corner_ele.value))
    return TriangleElementType.b_edge
  if C_corner_ele.value is None :
    sinB = (2*S_ele.value)/(a_edge_ele.value * b_edge_ele.value)
    C_corner_ele.value = math.asin(sinB)
    return TriangleElementType.C_corner
  return None

# 4. Công thức tỉ số giữa một cạnh và sin của góc đối diện với cạnh đó bằng đường kính của đường tròn ngoại tiếp tam giác. a/SinA = b/SinB= C/SinC = 2R

In [22]:
# a/sinA = b/Sinb
def RatioEdgeAndCornerA(currentElements:  Dict[TriangleElementType, TriangleElement]):
  a_edge_ele = currentElements[TriangleElementType.a_edge]
  b_edge_ele = currentElements[TriangleElementType.b_edge]
  A_corner_ele = currentElements[TriangleElementType.A_corner]
  B_corner_ele = currentElements[TriangleElementType.B_corner]

  if a_edge_ele is None or b_edge_ele is None or A_corner_ele is None or B_corner_ele is None:
    return None

  countEleNoVal = sum([1 for x in [a_edge_ele, b_edge_ele, A_corner_ele, B_corner_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if a_edge_ele.value is None :
    a_edge_ele.value = (b_edge_ele.value * math.sin(A_corner_ele.value))/math.sin(B_corner_ele.value)
    return TriangleElementType.a_edge
  if b_edge_ele.value is None :
    b_edge_ele.value = (a_edge_ele.value * math.sin(B_corner_ele.value))/math.sin(A_corner_ele.value)
    return TriangleElementType.b_edge
  if A_corner_ele.value is None :
    sinA = (a_edge_ele.value * math.sin(B_corner_ele.value))/ b_edge_ele.value
    A_corner_ele.value = math.asin(sinA)
    return TriangleElementType.A_corner
  if B_corner_ele.value is None :
    sinB = (b_edge_ele.value * math.sin(A_corner_ele.value))/ a_edge_ele.value
    B_corner_ele.value = math.asin(sinB)
    return TriangleElementType.B_corner
  return None

In [23]:
# b/SinB = c/SinC
def RatioEdgeAndCornerB(currentElements:  Dict[TriangleElementType, TriangleElement]):
  b_edge_ele = currentElements[TriangleElementType.b_edge]
  c_edge_ele = currentElements[TriangleElementType.C_edge]
  B_corner_ele = currentElements[TriangleElementType.B_corner]
  C_corner_ele = currentElements[TriangleElementType.C_corner]

  if b_edge_ele is None or c_edge_ele is None or B_corner_ele is None or C_corner_ele is None:
    return None

  countEleNoVal = sum([1 for x in [b_edge_ele, c_edge_ele, B_corner_ele, C_corner_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if b_edge_ele.value is None :
    b_edge_ele.value = (c_edge_ele.value * math.sin(B_corner_ele.value))/math.sin(C_corner_ele.value)
    return TriangleElementType.b_edge
  if c_edge_ele.value is None :
    c_edge_ele.value = (b_edge_ele.value * math.sin(C_corner_ele.value))/math.sin(B_corner_ele.value)
    return TriangleElementType.c_edge
  if B_corner_ele.value is None :
    sinB = (b_edge_ele.value * math.sin(C_corner_ele.value))/ c_edge_ele.value
    B_corner_ele.value = math.asin(sinB)
    return TriangleElementType.B_corner
  if C_corner_ele.value is None :
    sinC = (c_edge_ele.value * math.sin(B_corner_ele.value))/ b_edge_ele.value
    C_corner_ele.value = math.asin(sinC)
    return TriangleElementType.C_corner
  return None

In [24]:
# c/SinC = a/SinA
def RatioEdgeAndCornerC(currentElements:  Dict[TriangleElementType, TriangleElement]):
  c_edge_ele = currentElements[TriangleElementType.c_edge]
  a_edge_ele = currentElements[TriangleElementType.a_edge]
  C_corner_ele = currentElements[TriangleElementType.C_corner]
  A_corner_ele = currentElements[TriangleElementType.A_corner]

  if c_edge_ele is None or a_edge_ele is None or C_corner_ele is None or A_corner_ele is None:
    return None

  countEleNoVal = sum([1 for x in [c_edge_ele, a_edge_ele, C_corner_ele, A_corner_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if c_edge_ele.value is None :
    c_edge_ele.value = (a_edge_ele.value * math.sin(C_corner_ele.value))/math.sin(A_corner_ele.value)
    return TriangleElementType.c_edge
  if a_edge_ele.value is None :
    a_edge_ele.value = (c_edge_ele.value * math.sin(A_corner_ele.value))/math.sin(C_corner_ele.value)
    return TriangleElementType.a_edge
  if C_corner_ele.value is None :
    sinC = (c_edge_ele.value * math.sin(A_corner_ele.value))/ a_edge_ele.value
    C_corner_ele.value = math.asin(sinC)
    return TriangleElementType.C_corner
  if A_corner_ele.value is None :
    sinA = (a_edge_ele.value * math.sin(C_corner_ele.value))/ c_edge_ele.value
    A_corner_ele.value = math.asin(sinA)
    return TriangleElementType.A_corner
  return None

# 5. Nửa chu vi của tam giác: p = 1/2 (a + b + c)

In [25]:
# p = 1/2 abc
def HalfPerimeter(currentElements:  Dict[TriangleElementType, TriangleElement]):
  a_edge_ele = currentElements[TriangleElementType.a_edge]
  b_edge_ele = currentElements[TriangleElementType.b_edge]
  c_edge_ele = currentElements[TriangleElementType.c_edge]
  p_ele = currentElements[TriangleElementType.p]

  if a_edge_ele is None or b_edge_ele is None or c_edge_ele is None or p_ele is None:
    return None

  countEleNoVal = sum([1 for x in [a_edge_ele, b_edge_ele, c_edge_ele, p_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if a_edge_ele.value is None :
    a_edge_ele.value = 2*p_ele.value - b_edge_ele.value - c_edge_ele.value
    return TriangleElementType.a_edge
  if b_edge_ele.value is None :
    b_edge_ele.value = 2*p_ele.value - a_edge_ele.value - c_edge_ele.value
    return TriangleElementType.b_edge
  if c_edge_ele.value is None :
    c_edge_ele.value = 2*p_ele.value - a_edge_ele.value - b_edge_ele.value
    return TriangleElementType.c_edge
  if p_ele.value is None :
    p_ele.value = 0.5 * (a_edge_ele.value + b_edge_ele.value + c_edge_ele.value)
    return TriangleElementType.p
  return None

# 6. Công thức He-rong tính diện tích tam giác: S^2 = p(p-a).(p-b).(p-c)

In [26]:
# he-rong S = sqrt[p(p-a)(p-b)(p-c)]
def Herong(currentElements:  Dict[TriangleElementType, TriangleElement]):
  a_edge_ele = currentElements[TriangleElementType.a_edge]
  b_edge_ele = currentElements[TriangleElementType.b_edge]
  c_edge_ele = currentElements[TriangleElementType.c_edge]
  p_ele = currentElements[TriangleElementType.p]
  S_ele = currentElements[TriangleElementType.S]

  if a_edge_ele is None or b_edge_ele is None or c_edge_ele is None or p_ele is None or S_ele is None:
    return None

  countEleNoVal = sum([1 for x in [a_edge_ele, b_edge_ele, c_edge_ele, p_ele, S_ele] if x.value is None])
  if countEleNoVal != 1:
    return None

  if S_ele.value is None :
    s_2 = p_ele.value * (p_ele.value - a_edge_ele.value) * (p_ele.value - b_edge_ele.value) * (p_ele.value - c_edge_ele.value)
    S_ele.value = math.sqrt(s_2)
    return TriangleElementType.S
  if a_edge_ele.value is None:
    s_2 = S_ele.value ** 2
    p = p_ele.value
    b = b_edge_ele.value
    c = c_edge_ele.value
    a_edge_ele.value = p - s_2/(p*(p-a)*(p-b))
    return TriangleElementType.a_edge
  if b_edge_ele.value is None:
    s_2 = S_ele.value ** 2
    p = p_ele.value
    a = a_edge_ele.value
    c = c_edge_ele.value
    b_edge_ele.value = p - s_2/(p*(p-a)*(p-c))
    return TriangleElementType.b_edge
  if c_edge_ele.value is None:
    s_2 = S_ele.value ** 2
    p = p_ele.value
    a = a_edge_ele.value
    b = c_edge_ele.value
    c_edge_ele.value = p - s_2/(p*(p-a)*(p-b))
    return TriangleElementType.c_edge
  return None

# Thực thi

In [32]:
# Danh sách công thức
sumCorners = TriangleFormular('SumConers', 'A+B+C=Pi', [TriangleElementType.A_corner, TriangleElementType.B_corner, TriangleElementType.C_corner], SumCorners)
areaFromHeightA = TriangleFormular('AreaFromHeightA', 'S=1/2 a.hA', [TriangleElementType.S, TriangleElementType.a_edge, TriangleElementType.hA], AreaFromHeightA)
areaFromHeightB = TriangleFormular('AreaFromHeightB', 'S=1/2 b.hB', [TriangleElementType.S, TriangleElementType.b_edge, TriangleElementType.hB], AreaFromHeightB)
areaFromHeightC = TriangleFormular('AreaFromHeightC', 'S=1/2 c.hC', [TriangleElementType.S, TriangleElementType.c_edge, TriangleElementType.hC], AreaFromHeightC)
areaFromSinA = TriangleFormular('AreaFromSinA', 'S=1/2 bc.sinA', [TriangleElementType.S, TriangleElementType.b_edge, TriangleElementType.c_edge, TriangleElementType.A_corner], AreaFromSinA)
areaFromSinB = TriangleFormular('AreaFromSinB', 'S=1/2 ac.sinB', [TriangleElementType.S, TriangleElementType.a_edge, TriangleElementType.c_edge, TriangleElementType.B_corner], AreaFromSinB)
areaFromSinC = TriangleFormular('AreaFromSinC', 'S=1/2 ab.sinC', [TriangleElementType.S, TriangleElementType.a_edge, TriangleElementType.b_edge, TriangleElementType.C_corner], AreaFromSinC)
ratioEdgeAndCornerA = TriangleFormular('RatioEdgeAndCornerA', 'a/SinA = b/SinB', [TriangleElementType.a_edge, TriangleElementType.A_corner, TriangleElementType.b_edge, TriangleElementType.B_corner], RatioEdgeAndCornerA)
ratioEdgeAndCornerB = TriangleFormular('RatioEdgeAndCornerB', 'b/SinB = c/SinC', [TriangleElementType.b_edge, TriangleElementType.B_corner, TriangleElementType.c_edge, TriangleElementType.C_corner], RatioEdgeAndCornerB)
ratioEdgeAndCornerC = TriangleFormular('RatioEdgeAndCornerC', 'c/SinC = a/SinA', [TriangleElementType.c_edge, TriangleElementType.C_corner, TriangleElementType.a_edge, TriangleElementType.A_corner], RatioEdgeAndCornerC)
halfPerimeter = TriangleFormular('HalfPerimeter', 'p=1/2 (a+b+c)', [TriangleElementType.a_edge, TriangleElementType.b_edge, TriangleElementType.c_edge, TriangleElementType.p], HalfPerimeter)
herong = TriangleFormular('He-rong', 'S^2 = p(p-a)(p-b)(p-c)', [TriangleElementType.a_edge, TriangleElementType.b_edge, TriangleElementType.c_edge, TriangleElementType.p, TriangleElementType.S], Herong)

triangleNetwork = TriangleKnowledgeNetwork([sumCorners, areaFromHeightA, areaFromHeightB, areaFromHeightC, areaFromSinA, areaFromSinB,
                                            areaFromSinC, ratioEdgeAndCornerA, ratioEdgeAndCornerB, ratioEdgeAndCornerC, halfPerimeter, herong])


In [36]:
# Tìm các yếu tố còn lại của tam giác khi biết 3 cạnh a, b, c
a_edge_ele =TriangleElement('a', elementType = TriangleElementType.a_edge)
a_edge_ele.value = 3
b_edge_ele =TriangleElement( 'b', elementType = TriangleElementType.b_edge)
b_edge_ele.value = 4
c_edge_ele =TriangleElement('c', elementType = TriangleElementType.c_edge)
c_edge_ele.value = 5

currents : Dict[TriangleElementType, TriangleElement]  =  {}
currents[TriangleElementType.a_edge] = a_edge_ele
currents[TriangleElementType.b_edge] = b_edge_ele
currents[TriangleElementType.c_edge] = c_edge_ele

matrixAfter, fullDict = triangleNetwork.propagationActivationAlg(currents, False)
for i in fullDict:
  print(fullDict[i])

a_edge(3)
b_edge(4)
c_edge(5)
A_corner(0.6435011087932844)
B_corner(0.9272952180016123)
C_corner(1.5707963267948966)
S(6.0)
p(6.0)
hA(4.0)
hB(3.0)
hC(2.4)
